In [1]:
#! /usr/bin/env python

'''
Trains 7D QuaLiKiz-NN with a single output (efeETG_GB)
'''

from __future__ import print_function

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import RMSprop, adam, Adam
from keras.initializers import TruncatedNormal
from keras import regularizers
from keras import backend as K
import pandas
import numpy
import sys
import os

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from copy import deepcopy
from keras.models import load_model

Using TensorFlow backend.


In [9]:
# Define new Metric: rmse = Root Mean Square Error
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square( y_true-y_pred )))

# Gets the current file name. Useful for procedurally generating output/log files.
file_name =  os.path.basename(sys.argv[0][:-3])

# Define neural network parameters
batch_size = 10
#num_classes = 1
epochs = 100

# Load Data (which is in HDF5 or .h5 format)
store = pandas.HDFStore("../unstable_training_gen3_7D_nions0_flat_filter8.h5")
target_df = store['/output/efeETG_GB'].to_frame()  # This one is relatively easy to train
input_df = store['input']

# print(store.shape)

for key in store.keys():
    print(key) #Names of the groups in HDF5 file.

/constants
/flattened
/input
/output/ITG
/output/TEM
/output/absambi
/output/cke
/output/cki
/output/dfeITG_GB
/output/dfeITG_GB_div_efiITG_GB
/output/dfeTEM_GB
/output/dfeTEM_GB_div_efeTEM_GB
/output/dfe_GB
/output/dfiITG_GB
/output/dfiITG_GB_div_efiITG_GB
/output/dfiTEM_GB
/output/dfiTEM_GB_div_efeTEM_GB
/output/dfi_GB
/output/efeETG_GB
/output/efeITG_GB
/output/efeITG_GB_div_efiITG_GB
/output/efeTEM_GB
/output/efe_GB
/output/efiITG_GB
/output/efiTEM_GB
/output/efiTEM_GB_div_efeTEM_GB
/output/efi_GB
/output/gam_great_GB
/output/gam_leq_GB
/output/pfeITG_GB
/output/pfeITG_GB_div_efiITG_GB
/output/pfeTEM_GB
/output/pfeTEM_GB_div_efeTEM_GB
/output/pfe_GB
/output/pfiITG_GB
/output/pfiTEM_GB
/output/pfi_GB
/output/vceITG_GB
/output/vceITG_GB_div_efiITG_GB
/output/vceTEM_GB
/output/vceTEM_GB_div_efeTEM_GB
/output/vce_GB
/output/vciITG_GB
/output/vciITG_GB_div_efiITG_GB
/output/vciTEM_GB
/output/vciTEM_GB_div_efeTEM_GB
/output/vci_GB
/output/vfiITG_GB
/output/vfiTEM_GB
/output/vriITG_GB
/ou

In [8]:
input_df

,Ati,Ate,An,q,smag,x,Ti_Te
1,1.000000e-14,1.000000e-14,-5.0,0.66,-1.0,0.09,0.25
7,1.000000e-14,1.000000e-14,-5.0,0.66,-1.0,0.09,0.50
13,1.000000e-14,1.000000e-14,-5.0,0.66,-1.0,0.09,0.75
25,1.000000e-14,1.000000e-14,-5.0,0.66,-1.0,0.09,1.33
31,1.000000e-14,1.000000e-14,-5.0,0.66,-1.0,0.09,1.66
37,1.000000e-14,1.000000e-14,-5.0,0.66,-1.0,0.09,2.50
43,1.000000e-14,1.000000e-14,-5.0,0.66,-1.0,0.21,0.25
61,1.000000e-14,1.000000e-14,-5.0,0.66,-1.0,0.21,1.00
67,1.000000e-14,1.000000e-14,-5.0,0.66,-1.0,0.21,1.33
73,1.000000e-14,1.000000e-14,-5.0,0.66,-1.0,0.21,1.66


In [10]:
target_df

,efeETG_GB
978721,1.698403
981745,0.848584
981787,2.298091
985063,0.492831
985105,2.069086
985147,2.804335
988423,0.936546
988465,2.947697
988501,0.558062
988507,2.746332
